In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt; plt.rcdefaults()
from IPython.display import display
import io
import sys
import arrow
import datetime
import os
import graphlab as gl

In [3]:
os.getcwd()

'/Users/phamthailinh/Desktop/upgithub/My_extended_side_project'

In [196]:
data = pd.read_csv("/Users/phamthailinh/Desktop/upgithub/My_extended_side_project/test.csv")

In [197]:
data = gl.SFrame(data = data)

In [198]:
data['word_count'] = gl.text_analytics.count_words(data['sentence'])
data.head(5)

Unnamed: 0,sentence,frequencies,word_count
0,"hello, Sub-Periods, he isLi hey ^^ ...",1,"{'hello,': 1, '^^': 1,'is': 1, 'sub-periods,': ..."
1,hello a 😊,2,"{'a': 1,'\xf0\x9f\x98\x8a': 1, ..."
2,You can now call eachother and see informa ...,1,"{'status': 1,'information': 1, 'and': ..."
3,v see at the right,2,"{'the': 1, 'right': 1,'at': 1, 'see': 1, 'v': ..."
4,Hi e,1,"{'hi': 1, 'e': 1}"


In [199]:
data['sentiment'] = data['frequencies'] == 1
data.head(5)

Unnamed: 0,sentence,frequencies,word_count,sentiment
0,"hello, Sub-Periods, he isLi hey ^^ ...",1,"{'hello,': 1, '^^': 1,'is': 1, 'sub-periods,': ...",1
1,hello a 😊,2,"{'a': 1,'\xf0\x9f\x98\x8a': 1, ...",0
2,You can now call eachother and see informa ...,1,"{'status': 1,'information': 1, 'and': ...",1
3,v see at the right,2,"{'the': 1, 'right': 1,'at': 1, 'see': 1, 'v': ...",0
4,Hi e,1,"{'hi': 1, 'e': 1}",1


In [200]:
gl.canvas.set_target('ipynb')

In [207]:
train_data, test_data = data.random_split(.9, seed=0)

In [209]:
sentiment_model = gl.logistic_classifier.create(train_data,
                                                target='sentiment',
                                                features=['word_count'],
                                                validation_set=test_data)

WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

Logistic regression:

--------------------------------------------------------

Number of examples          : 1266

Number of classes           : 2

...

| 5         | 8        | 1.000000  | 0.044789     | 0.983412          | 0.816327            |

| 6         | 9        | 1.000000  | 0.052541     | 0.988152          | 0.809524            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [210]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful']

In [211]:
def word_count(word, word_dict): 
    if word in word_dict : return word_dict[word] 
    return 0
for word in selected_words:
    data[word] = data['word_count'].apply(lambda word_dict : word_count(word, word_dict))

In [212]:
train_data,test_data = data.random_split(.8, seed=0)

In [213]:
selected_words_model = gl.logistic_classifier.create(train_data,
                                                    target='sentiment',
                                                    features=selected_words,
                                                    validation_set=test_data)

WARNING: Detected extremely low variance for feature(s) 'awesome', 'fantastic', 'amazing', 'horrible', 'terrible' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.

Logistic regression:

--------------------------------------------------------

Number of examples          : 1121

Number of classes           : 2

...

| 4         | 5        | 0.004965     | 0.897413          | 0.852740            |

| 5         | 6        | 0.006464     | 0.897413          | 0.852740            |

| 6         | 7        | 0.007968     | 0.897413          | 0.852740            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

In [214]:
selected_words_model.evaluate(test_data, metric='roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+-----+-----+-----+----+
 | threshold | fpr | tpr |  p  | n  |
 +-----------+-----+-----+-----+----+
 |    0.0    | 1.0 | 1.0 | 249 | 43 |
 |   1e-05   | 1.0 | 1.0 | 249 | 43 |
 |   2e-05   | 1.0 | 1.0 | 249 | 43 |
 |   3e-05   | 1.0 | 1.0 | 249 | 43 |
 |   4e-05   | 1.0 | 1.0 | 249 | 43 |
 |   5e-05   | 1.0 | 1.0 | 249 | 43 |
 |   6e-05   | 1.0 | 1.0 | 249 | 43 |
 |   7e-05   | 1.0 | 1.0 | 249 | 43 |
 |   8e-05   | 1.0 | 1.0 | 249 | 43 |
 |   9e-05   | 1.0 | 1.0 | 249 | 43 |
 +-----------+-----+-----+-----+----+
 [100001 rows x 5 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.}

In [215]:
selected_words_model.show(view='Evaluation')

In [216]:
data[(data['great'] == 1) | (data['love'] == 1) | (data['bad'] == 1)].head(3)

Columns:
	Unnamed: 0	int
	sentence	str
	frequencies	int
	word_count	dict
	sentiment	int
	awesome	int
	great	int
	fantastic	int
	amazing	int
	love	int
	horrible	int
	bad	int
	terrible	int
	awful	int

Rows: 3

Data:
+------------+-------------------------------+-------------+
| Unnamed: 0 |            sentence           | frequencies |
+------------+-------------------------------+-------------+
|     64     |      s love it the most v     |      1      |
|    115     | e also love to see some ty... |      1      |
|    415     | Flight delay, back to sg a... |      1      |
+------------+-------------------------------+-------------+
+-------------------------------+-----------+---------+-------+-----------+---------+
|           word_count          | sentiment | awesome | great | fantastic | amazing |
+-------------------------------+-----------+---------+-------+-----------+---------+
| {'love': 1, 'it': 1, 'most... |     1     |    0    |   0   |     0     |    0    |
| {'some': 1, 'afraid': 1, '... |     1     |    0    |   0   |     0     |    0    |
| {'few': 1, 'because': 1, '... |     1     |    0    |   0   |     0     |    0    |
+-------------------------------+-----------+---------+-------+-----------+---------+
+------+----------+-----+----------+-------+
| love | horrible | bad | terrible | awful |
+------+----------+-----+----------+-------+
|  1   |    0     |  0  |    0     |   0   |
|  1   |    0     |  0  |    0     |   0   |
|  0   |    0     |  1  |    0     |   0   |
+------+----------+-----+----------+-------+
[3 rows x 14 columns]

In [219]:
# calcutelate the pos/ total signed expressions.
# and the proportion pos/ neg
sum_positive = 0
sum_negative = 0
for word in selected_words :
    if word == 'awesome' or word == 'great' or word == 'fantastic' or word == 'love':
        sum_positive += data[word].sum()
    elif word == 'horrible' or word == 'bad' or word == 'terrible' or word == 'awful':
        sum_negative += data[word].sum()
sum_sentiment = sum_positive + sum_negative
proportion = sum_positive*1.0/sum_negative
positive_expression = sum_positive*1.0/sum_sentiment*100
print('Percent of the positive expression is {}%'.format(round(positive_expression,1)))
print('The proportion between positive and negative sentiment is: {}'.format(round(proportion,1)))

Percent of the positive expression is 62.5%
The proportion between positive and negative sentiment is: 1.7
